## GC常见问法

### 蚂蚁金服
1. 你知道哪几种垃圾回收器，各自的优缺点，重点讲一下CMS和G1？
2. JVM GC算法有哪些，目前的JDK版本采用什么回收算法？
3. G1回收器讲下回收过程GC是什么？为什么要有GC？
4. GC的两种判定方法？CMS收集器与G1收集器的特点
### 百度
1. 说一下GC算法，分代回收说下
2. 垃圾收集策略和算法
### 天猫
1. JVM GC原理，JVM怎么回收内存
2. CMS特点，垃圾回收算法有哪些？各自的优缺点，他们共同的缺点是什么？
### 滴滴
1. Java的垃圾回收器都有哪些，说下G1的应用场景，平时你是如何搭配使用垃圾回收器的
### 京东
1. 你知道哪几种垃圾收集器，各自的优缺点，重点讲下CMS和G1，
2. 包括原理，流程，优缺点。垃圾回收算法的实现原理
### 阿里
1. 讲一讲垃圾回收算法。
2. 什么情况下触发垃圾回收？
3. 如何选择合适的垃圾收集算法？
4. JVM有哪三种垃圾回收器？
### 字节跳动
1. 常见的垃圾回收器算法有哪些，各有什么优劣？
2. System.gc()和Runtime.gc()会做什么事情？
3. Java GC机制？GC Roots有哪些？
4. Java对象的回收方式，回收算法。
5. CMS和G1了解么，CMS解决什么问题，说一下回收的过程。
6. CMS回收停顿了几次，为什么要停顿两次?

## GC垃圾回收相关

### 0. 什么是GC，为什么要GC

### 0.0 对象的finalization机制

由于finalize()方法的存在，虚拟机中的对象一般处于**三种可能的状态。**

* 如果从所有的根节点都无法访问到某个对象，说明对象己经不再使用了。一般来说，此对象需要被回收。但事实上，也并非是“非死不可”的，这时候它们暂时处于“缓刑”阶段。一个无法触及的对象有可能在某一个条件下“复活”自己，如果这样，那么对它立即进行回收就是不合理的。为此，定义虚拟机中的对象可能的三种状态。如下：
    * **可触及的**：从根节点开始，可以到达这个对象。
    * **可复活的**：对象的所有引用都被释放，但是对象有可能在finalize()中复活。
    * **不可触及的**：对象的finalize()被调用，并且没有复活，那么就会进入不可触及状态。不可触及的对象不可能被复活，因为finalize()只会被调用一次。
* 只有在对象不可触及时才可以被回收。

**判定一个对象objA是否可回收，至少要经历两次标记过程：**

* 如果对象objA到GC Roots没有引用链，则进行第一次标记。
* 进行筛选，判断此对象是否有必要执行finalize()方法
    * 如果对象objA没有重写finalize()方法，或者finalize()方法已经被虚拟机调用过，则虚拟机视为“没有必要执行”，objA被判定为不可触及的。
    * 如果对象objA重写了finalize()方法，且还未执行过，那么objA会被插入到F-Queue队列中，由一个虚拟机自动创建的、低优先级的Finalizer线程触发其finalize()方法执行。
* finalize()方法是对象逃脱死亡的最后机会，稍后GC会对F-Queue队列中的对象进行第二次标记。如果objA在finalize()方法中与引用链上的任何一个对象建立了联系，那么在第二次标记时，objA会被移出“即将回收”集合。之后，对象会再次出现没有引用存在的情况。在这个情况下，finalize()方法不会被再次调用，对象会直接变成不可触及的状态，也就是说，**一个对象的finalize()方法只会被调用一次。**

### 0.1 再谈引用——强软弱虚

1. **强引用**（StrongReference）：最传统的“引用”的定义，是指在程序代码之中普遍存在的引用赋值，即类似“object obj=new Object()”这种引用关系。无论任何情况下，只要强引用关系还存在，垃圾收集器就永远不会回收掉被引用的对象。宁可报OOM，也不会GC强引用
2. **软引用**（SoftReference）：用于维护一些可有可无的对象；在系统将要发生内存溢出之前，将会把这些对象列入回收范围之中进行第二次回收。如果这次回收后还没有足够的内存，才会抛出内存溢出异常。（内存不足即回收）
3. **弱引用**（WeakReference）：被弱引用关联的对象只能生存到下一次垃圾收集之前。当垃圾收集器工作时，无论内存空间是否足够，都会回收掉被弱引用关联的对象。（发现即回收）
4. **虚引用**（PhantomReference）：一个对象是否有虚引用的存在，完全不会对其生存时间构成影响，也无法通过虚引用来获得一个对象的实例。为一个对象设置虚引用关联的唯一目的就是能在这个对象被收集器回收时收到一个系统通知。

### 0.2 System.gc()与Runtime.gc()

调用 System.gc() 实际上等效于调用： Runtime.getRuntime().gc()

## 1. 垃圾回收算法——标记+回收

### 1.1 标记阶段算法

* **引用计数算法**：在每个对象实例中保存一个**引用计数器**属性，用于记录对象实例被引用的次数，每当增加一个引用，计数器+1，引用失效时，计数器-1；当引用计数器的直为0时，表明该对象实例没有被使用，可以进行回收。
* 使用情况：python等


* **优点**：实现简单；判别效率高；回收没有延迟
* **缺点**：
    1. 存储额外字段有内存开销；
    2. 引用计数器的计算有时间开销；
    3. 不能回收循环引用的情况

* **循环引用**：
* **解决**
    1. 手动解除，在合适的时机，解除引用关系
    2. 使用弱引用；（当对象A引用B使用强引用时，B引用A则应使用弱引用）

![](img/jvm-rr.png)

* **可达性分析算法**：根搜索算法，以**根对象(GC roots)** 作为起点，从上到下搜索被根对象关联的节点是否可达，从根节点出发所有直接或间接被搜索到的节点会形成一个引用链，表明这些对象是存活的；没有搜索到的节点，说明对象不可达，应该被回收
* 使用情况：java, c#等

**可以作为GC Roots的对象**
1. 虚拟机栈中引用的对象  
    * 比如：各个线程被调用的方法中使用到的参数、局部变量等。
2. 本地方法栈内JNI（通常说的本地方法）引用的对象
3. 方法区中类静态属性引用的对象
    * 比如：Java类的引用类型静态变量
4. 方法区中常量引用的对象
    * 比如：字符串常量池（StringTable）里的引用
5. 所有被同步锁synchronized持有的对象
6. Java虚拟机内部的引用。
    * 基本数据类型对应的Class对象，一些常驻的异常对象（如：NullPointerException、OutofMemoryError），系统类加载器。
7. 反映java虚拟机内部情况的JMXBean、JVMTI中注册的回调、本地代码缓存等。

* 总结：除了堆空间的周边，比如：虚拟机栈、本地方法栈、方法区、字符串常量池等地方对堆空间进行引用的，都可以作为GC Roots进行可达性分析
* 除了这些固定的GC Roots集合以外，根据用户所选用的垃圾收集器以及当前回收的内存区域不同，还可以有其他对象**“临时性”** 地加入，共同构成完整GC Roots集合。比如：分代收集和局部回收（PartialGC）。
    * 如果只针对Java堆中的某一块区域进行垃圾回收（比如：典型的只针对新生代），必须考虑到内存区域是虚拟机自己的实现细节，更不是孤立封闭的，这个区域的对象完全有可能被其他区域的对象所引用，这时候就需要一并将关联的区域对象也加入GC Roots集合中去考虑，才能保证可达性分析的准确性。

### 1.2 回收算法

* **复制算法**：
* 将内存区域划分成2块，每次只使用其中的一块；
* 当进行垃圾回收时，将正在使用的那一块内存中存活的对象复制到另外一块中；然后直接清空；
* 依次交替

* **优点**：
    1. 实现简单，运行高效
    2. 复制过去以后保证空间的连续性，不会出现“碎片”问题。
* **缺点**：
    1. 浪费空间，需要两倍的内存空间。
    2. 对象复制到另外地址后，需要维护对象引用关系

* **标记清除算法**：
* 先使用可达性分析算法标记存活对象；
* 再遍历一次，将没有标记的对象进行清除；
* 所谓的清除并不是真的置空，而是把需要清除的对象地址保存在空闲的地址列表里。下次有新对象需要加载时，判断垃圾的位置空间是否够，如果够，就存放（也就是覆盖原有的地址）。

* **优点**：
    1. 

* **缺点**：
    1. 效率不算高
    2. 在进行GC的时候，需要停止整个应用程序，用户体验较差（STW）
    3. 这种方式清理出来的空闲内存是不连续的，产生内碎片，需要维护一个空闲列表

* **标记整理/压缩算法**
* 先使用可达性分析算法堆存活对象进行标记
* 再遍历一次，将所有存活的对象压缩到内存的一端，再清理剩下的空间

* **优点优点**
    1. 消除了标记-清除算法当中，内存区域分散的缺点，我们需要给新对象分配内存时，JVM只需要持有一个内存的起始地址即可。  
    
    2. 消除了复制算法当中，内存减半的高额代价。

* **缺点**

    1. 从效率上来说，标记-整理算法要低于复制算法。
    2. 移动对象的同时，如果对象被其他对象引用，则还需要调整引用的地址（因为HotSpot虚拟机采用的不是句柄池的方式，而是直接指针）
    3. 移动过程中，需要全程暂停用户应用程序。即：STW


* **分代收集算法**
* 没有使用新的算法，只是根据堆区不同的划分使用不同的算法

### 几种垃圾回收算法的比较

![](img/jvm-al-cp.png)

## 2. 垃圾回收器

![](img/jvm-gc.jpg)

### 2.1 串行回收器

* **Serial GC**（新生代）
* 这个收集器是一个单线程的收集器，只会使用一个CPU（串行）或一条收集线程去完成垃圾收集工作。在它进行垃圾收集时，必须暂停其他所有的工作线程，直到它收集结束（Stop The World）；
* 这是hotspot client模式下默认的垃圾回收器
* 使用的是**复制算法**


* **优点**


1. 简单而高效（与其他收集器的单线程比），对于限定单个CPU的环境来说，Serial收集器由于**没有线程交互的开销**，专心做垃圾收集自然可以获得最高的单线程收集效率。运行在Client模式下的虚拟机是个不错的选择。
2. 在用户的桌面应用场景中，可用内存一般不大（几十MB至一两百MB），可以在较短时间内完成垃圾收集（几十ms至一百多ms），只要不频繁发生，使用串行回收器是可以接受的。

* **Serial Old GC** （老年代）
* 使用在老年代的垃圾回收器，串行，使用的是**标记压缩算法**
* 同样有STW

在GC执行过程都是暂停用户线程，然后GC线程独占执行复制算法或者标记压缩算法

### 2.2 并行回收器

* **ParNew GC** （新生代）
* 与Serial几乎没有区别
* 它使用的是并行，多线程
* **复制算法**

* **Parallel Old GC** （老年代）
* **标记压缩算法**
* 并行，多线程

* **Parallel Scavenge GC** （新生代）——目标：实现**吞吐量可控**（吞吐量优先）
* **复制算法**
* 与ParNew GC类似，使用的也是并行，多线程，只是Parallel Scavenge GC更多关注虚拟机的吞吐量；
* -XX： MaxGCPauseMillis 控制GC的最大时长（也就是进行一次GC STW的时长）
* -XX： GCTimeRatio 控垃圾收集时间占总时间的比例

高吞吐量则可以高效率地利用CPU时间，尽快完成程序的运算任务，**主要适合在后台运算而不需要太多交互的任务**。因此，常见在服务器环境中使用。例如，那些执行批量处理、订单处理、工资支付、科学计算的应用程序

### 2.3 并发回收器

### **CMS GC** （老年代）
* 并发垃圾回收器，实现了让垃圾收集线程与用户线程同时工作；
* 目标：**低延迟**，尽可能缩短垃圾收集时用户线程的停顿时间，适合用在互联网站或者B/S系统的服务端上，这类应用尤其重视服务的响应速度，希望系统停顿时间最短，以给用户带来较好的体验。CMS收集器就非常符合这类应用的需求。
* **标记清除算法**

### CMS工作原理/过程——4个步骤，2个暂停

![](img/jvm-cms.png)

1. **初始标记**（CMS initial mark）：仅仅只是标记一下GC Roots能直接关联到的对象， 速度很快；
2. **并发标记**（CMS concurrent mark）：从GC Roots的直接关联对象开始遍历整个对象图的过程， 这个过程耗时较长，用户线程与垃圾回收线程并发执行；
3. **重新标记**（CMS remark）：修正并发标记期间， 因用户程序继续运作而导致标记产生变动的那一部分对象的标记记录， 这个阶段的停顿时间通常会比初始标记阶段稍长一些， 但也远比并发标记阶段的时间短
4. **并发清除**（CMS concurrent sweep）：清理删除掉标记阶段判断的已经死亡的对象， 由于不需要移动存活对象， 所以这个阶段也是可以与用户线程同时并发的

* **由于最耗费时间的并发标记与并发清除阶段都不需要暂停工作，所以整体的回收是低停顿的**
* 当堆内存使用率达到某一阈值时，便开始进行回收，以确保应用程序在CMS工作过程中依然有足够的空间支持应用程序运行
* 要是CMS运行期间预留的内存无法满足程序需要，就会出现一次**$Concurrent Mode Failure$** 失败，这时虚拟机将启动后备预案：临时启用**Serial old**收集器来重新进行老年代的垃圾收集，这样停顿时间就很长了



* 为什么 CMS 不采用标记-压缩算法呢？
    * 因为当并发清除的时候，用Compact整理内存的话，原来的用户线程无法正确使用内；

* **优点**

    1. 并发收集
    2. 低延迟
* **弊端**

    1. **会产生内存碎片**，导致并发清除后，用户线程可用的空间不足。在无法分配大对象的情况下，不得不提前触发Full GC。
    2. CMS收集器**对CPU资源非常敏感**。在并发阶段，它虽然不会导致用户停顿，但是会因为占用了一部分线程而导致应用程序变慢，总吞吐量会降低。
    3. CMS收集器**无法处理浮动垃圾**。可能出现**$Concurrent Mode Failure$** 失败而导致另一次Full GC的产生。在并发标记阶段由于程序的工作线程和垃圾收集线程是同时运行或者交叉运行的，**那么在并发标记阶段如果产生新的垃圾对象，CMS将无法对这些垃圾对象进行标记，最终会导致这些新产生的垃圾对象没有被及时回收，**从而只能在下一次执行GC时释放这些之前未被回收的内存空间

### **G1 GC**（并行&并发， 整堆， 分区算法）——在延迟可控的情况下获得尽可能高的吞吐量

G1 垃圾回收器使用的是分区算法，它将整个堆内存划分成若干个相等的区域**Region**，每个区域可以作为**伊甸园区，幸存者区，老年代，巨型区**，其中伊甸园区、幸存者区与老年代与堆区分代是一样的，分配对象与是一样的；巨型区是用来存储大对象的，当一个对象需要分配的内存大于半个region时，这个对象就会被分配到巨型区，如果一个巨型区放不下，JVM会寻找一块连续的巨型区来存放（不然OOM）


* 对于需要回收的部分，G1 GC使用一个优先队列来记录回收每个Region的价值，**每次根据允许的收集时间，优先回收价值最大的Region**

![](img/jvm-g1-dd.png)

### G1回收的原理/过程

* **初始标记**（Initial Marking） ：**仅仅只是标记一下GC Roots能直接关联到的对象**， 并且修改TAMS（Next Top at Mark Start）指针的值， 让下一阶段用户线程并发运行时， 能正确地在可用的Region中分配新对象。 这个阶段需要停顿线程， 但耗时很短， 而且是借用进行Minor GC的时候同步完成的， 所以G1收集器在这个阶段实际并没有额外的停顿。**有STW**


* **并发标记**（Concurrent Marking） ： 从GC Root开始对堆中对象进行**可达性分析**， 递归扫描整个堆里的对象图， 找出要回收的对象， 这阶段耗时较长， 但可**与用户程序并发执行**。 当对象图扫描完成以后， 还要重新处理SATB记录下的在并发时有引用变动的对象。


* **最终标记**（Final Marking） ： 对用户线程做另一个短暂的暂停， 用于处理并发阶段结束后仍遗留下来的最后那少量的SATB（Snapshot At The Beginning）记录。（也就是处理在并发标记过程中发生变化的对象；**有STW**


* **筛选回收**（Live Data Counting and Evacuation） ： 负责更新Region的统计数据， 对各个Region的回收价值和成本进行排序， 根据用户所期望的停顿时间来制定回收计划， 可以自由选择任意多个Region构成回收集， 然后把决定回收的那一部分Region的存活对象复制到空的Region中， 再清理掉整个旧Region的全部空间。 这里的操作涉及存活对象的移动， 是必须暂停用户线程， 由多条收集器线程并行完成的。**有STW**

* 整个G1垃圾回收的过程可以分为2个阶段，一个是Yong GC，在这个阶段，只回收年轻代中的垃圾；也就是和之前所说的将伊甸园区的数据放入幸存者区（或者将幸存者区中的对象放入老年代
* 第二个阶段是，当堆内存使用量达到一定阈值（默认是45%）时，会开启混合回收，（在混合回收前还是有并发标记）；在混合回收阶段，会回收年轻代的一些区域，同时会将老年代一些价值比较高的区域进行回收；
* 当第二个阶段回收完毕后，又会回到第一阶段，循环往复；
* 如果垃圾回收没有提供足以使用的内存，会出发Full GC，使用的是Serial Old，会导致更长时间的STW


### 并发标记

* **解决的问题**：并发标记主要是为了解决STW时间过长的问题，通过gc线程与用户线程并发执行，减少对用户线程执行的影响

**三色标记**——原则：遍历对象图的时候必须在一个能保障一致性的快照中

* 在遍历对象图的过程中，把访问都的对象**按照"是否访问过"**这个条件标记成以下三种颜色：


* **白色：**表示对象尚**未被垃圾回收器访问过**。显然，在可达性分析刚刚开始的阶段，所有的对象都是白色的，若在分析结束的阶段，仍然是白色的对象，即代表不可达。


* **黑色：**表示对象**已经**被垃圾回收器访问过，且这个对象的**所有引用**都已经扫描过。黑色的对象代表已经扫描过，它是安全存活的，如果有其它的对象引用指向了黑色对象，无须重新扫描一遍。黑色对象不可能直接（不经过灰色对象）指向某个白色对象。


* **灰色：**表示对象已经被垃圾回收器访问过，但这个对象**至少存在一个引用还没有被扫描过**

![](img/jvm-cc-th.png)

并发标记在一致性快照的环境下进行标记是没有问题的，但是当进行并发标记的时候，就会出现**浮动垃圾** 与 **对象消失** 的问题

* **浮动垃圾**指的是：在此次标记过程中将原本应当消亡的对象标记成存活；这种情况还是可以容忍的，在下次垃圾回收的时候回收掉就可以了；
* **对象消失**指的是：在标记过程中，将存活的对象标记成死亡对象，这种情况是不可容忍的，会导致程序发生错误。

**对象消失**当且仅当以下2个条件同时发生的时候：其中一、二顺序不能颠倒  
* 一：赋值器插入了一条或多条**从黑色对象到白色**对象的新引用；
* 二：赋值器删除了全部从灰色对象到**该白色对象**的直接或间接引用。

**解决对象消失问题**——两种方案：**增量更新（Incremental Update）和原始快照（Snapshot At The Beginning，SATB）**
* CMS是基于增量更新来做并发标记的，G1则采用的是原始快照的方式
* 增量更新破坏的是条件一，原始快照SATB破坏的是条件二

**增量更新**
* 当黑色对象插入新的指向白色对象的引用关系时，就将这个新插入的引用记录下来，等并发扫描结束之后，再将这些记录过的引用关系中的黑色对象为根，重新扫描一次。

* 可以简化的理解为：黑色对象一旦插入了指向白色对象的引用之后，它就变回了灰色对象。

**原始快照（SATB）**
* **把旧的引用所指向的对象都变成非白的**
* 当灰色对象要删除指向白色对象的引用关系时，就将这个要删除的引用记录下来，在并发扫描结束之后，再将这些记录过的引用关系中的灰色对象为根，重新扫描一次。

* 这个可以简化理解为：无论引用关系删除与否，都会按照刚刚开始扫描那一刻的对象图快照开进行搜索

![](img/jvm-cc-stab.awebp)